# Лабораторная работа 1, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [28]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [29]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [30]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [31]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_11889/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
5334,0.275424,-0.014081,-0.105677,-0.997051,-0.980195,-0.972942,-0.997176,-0.979383,-0.971368,-0.942773,...,0.102889,-0.192463,0.266583,-0.205337,-0.900241,0.025413,0.01962,25,4,SITTING


Подготовим тестовую выборку

In [32]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_11889/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1800,0.288717,-0.02206,-0.092009,-0.37185,0.205228,-0.411859,-0.434948,0.128024,-0.384601,-0.049755,...,-0.316344,0.239978,0.378916,-0.537225,-0.713396,0.295043,-0.003574,13,1,WALKING


Поменяем названия функций

In [33]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [34]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

#### Обучим модель KNN

Функция чтения датасета

In [35]:
def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

Функция обучения KNN модели

In [36]:
def train_model(train_x, train_y, validation=None):
    model = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")       

Загрузим данные

In [37]:
train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

Классификация

In [38]:
model = train_model(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.9002375296912114
Recall = 0.9002375296912114
Precision = 0.9039585713237325
F1 Score = 0.8993037837295996


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)']+ [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn_regressor = KNeighborsRegressor(n_neighbors=5, weights='uniform', p=2, metric='manhattan')
knn_regressor.fit(X_train, y_train)

y_pred = knn_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")

MAE (CO2 Emissions): 3.93
MSE (CO2 Emissions): 76.53


## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.
2. Подбор гиперпараметров

    - Оптимизация количества соседей (n_neighbors): Ключевой гиперпараметр для KNN, который можно подобрать с использованием кросс-валидации. Также можно рассмотреть более адаптивные способы выбора соседей, например, весовые функции.
    - Выбор метрики расстояния: Проверить влияние различных метрик (евклидова, манхэттенского расстояния и т.д.) на результат классификации и регрессии.
    - Кросс-валидация: Проведение кросс-валидации с подбором гиперпараметров для нахождения оптимального n_neighbors, weights, metric и других гиперпараметров.

3. Методы оптимизации
    - Grid Search и Random Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели и Random Search для расширения возможностей поиска с более низкой вычислительной затратой.

### Обучение моделей, оценка качества обучения моделей по метрикам

#### Подбор гиперпараметров и Grid Search

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Гиперпараметры для классификации
param_grid_class = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_class = KNeighborsClassifier()
grid_search_class = GridSearchCV(knn_class, param_grid_class, cv=5, scoring='accuracy')
grid_search_class.fit(train_X, train_y)
best_knn_class = grid_search_class.best_estimator_

print("Лучшие параметры для классификации (HAR):", grid_search_class.best_params_)
print("Лучший результат на кросс-валидации (HAR):", grid_search_class.best_score_)

y_pred_class = best_knn_class.predict(test_X)
accuracy = accuracy_score(test_y, y_pred_class)
precision = precision_score(test_y, y_pred_class, average='weighted')
recall = recall_score(test_y, y_pred_class, average='weighted')
f1 = f1_score(test_y, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Гиперпараметры для регрессии
param_grid_reg = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_reg = KNeighborsRegressor()
grid_search_reg = GridSearchCV(knn_reg, param_grid_reg, cv=5, scoring='neg_mean_squared_error')
grid_search_reg.fit(X_train, y_train)
best_knn_reg = grid_search_reg.best_estimator_

print("Лучшие параметры для регрессии (CO2):", grid_search_reg.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_reg.best_score_)

y_pred_reg = best_knn_reg.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_reg))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_reg))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/User

Лучшие параметры для классификации (HAR): {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Лучший результат на кросс-валидации (HAR): 0.9816374625989077
Accuracy: 0.9049881235154394
Precision: 0.9083027317222394
Recall: 0.9049881235154394
F1 Score: 0.9040905399655608
Лучшие параметры для регрессии (CO2): {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Лучший результат на кросс-валидации (CO2): 45.26808079174798
MAE (CO2 Emissions): 2.3889615412605654
MSE (CO2 Emissions): 54.58617459908717


#### Random Search

In [41]:
from sklearn.model_selection import RandomizedSearchCV

random_search_class = RandomizedSearchCV(knn_class, param_grid_class, n_iter=10, cv=5, scoring='accuracy', random_state=0)
random_search_class.fit(train_X, train_y)
best_knn_class = random_search_class.best_estimator_
print("Лучшие параметры для классификации (HAR) с Random Search:", random_search_class.best_params_)
print("Лучший результат на кросс-валидации (HAR) с Random Search:", random_search_class.best_score_)

y_pred_class = best_knn_class.predict(test_X)
accuracy = accuracy_score(test_y, y_pred_class)
precision = precision_score(test_y, y_pred_class, average='weighted')
recall = recall_score(test_y, y_pred_class, average='weighted')
f1 = f1_score(test_y, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

random_search_reg = RandomizedSearchCV(knn_reg, param_grid_reg, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=0)
random_search_reg.fit(X_train, y_train)
best_knn_reg = random_search_reg.best_estimator_
print("Лучшие параметры для регрессии (CO2) с Random Search:", random_search_reg.best_params_)
print("Лучший результат на кросс-валидации (CO2) с Random Search:", -random_search_reg.best_score_)

y_pred_reg = best_knn_reg.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_reg))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_reg))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/User

Лучшие параметры для классификации (HAR) с Random Search: {'weights': 'distance', 'n_neighbors': 5, 'metric': 'manhattan'}
Лучший результат на кросс-валидации (HAR) с Random Search: 0.9785095057737575
Accuracy: 0.9134713267729895
Precision: 0.9171879851153816
Recall: 0.9134713267729895
F1 Score: 0.9126698790513678
Лучшие параметры для регрессии (CO2) с Random Search: {'weights': 'distance', 'n_neighbors': 3, 'metric': 'euclidean'}
Лучший результат на кросс-валидации (CO2) с Random Search: 46.201229542998945
MAE (CO2 Emissions): 2.395207429291693
MSE (CO2 Emissions): 48.30084075689549


#### Добавление нового признака (для задачи регрессии)

In [42]:
from sklearn.preprocessing import PolynomialFeatures

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

knn = KNeighborsRegressor()
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_poly, y_train_new_field)

best_knn = grid_search.best_estimator_

y_pred = best_knn.predict(X_test_poly)

mae = mean_absolute_error(y_test_new_field, y_pred)
mse = mean_squared_error(y_test_new_field, y_pred)

print("Лучшие гиперпараметры:", grid_search.best_params_)
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Лучшие гиперпараметры: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
MAE: 2.24
MSE: 34.31


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'weights': 'distance', 'n_neighbors': 5, 'metric': 'manhattan'}

In [43]:
def train_model(train_x, train_y, validation=None):
    model = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='manhattan')
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}") 

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')      

model = train_model(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.9134713267729895
Recall = 0.9134713267729895
Precision = 0.9171879851153816
F1 Score = 0.9126698790513678


#### Задача регрессии

Лучшие параметры для регрессии: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

knn_regressor = KNeighborsRegressor(n_neighbors=3, weights='distance', p=1)
knn_regressor.fit(X_train_poly, y_train_new_field)

y_pred = knn_regressor.predict(X_test_poly)

mae = mean_absolute_error(y_test_new_field, y_pred)
mse = mean_squared_error(y_test_new_field, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")

MAE (CO2 Emissions): 2.24
MSE (CO2 Emissions): 34.31


### Выводы

В задаче классификации на наборе данных HAR изначальная модель KNN показала хорошие результаты, однако оптимизация с помощью Grid Search позволила улучшить точность и F1-меру. Применение Манхэттенского расстояния и уменьшение числа соседей до трёх показали, что правильная настройка гиперпараметров помогает модели точнее классифицировать активности.

В задаче регрессии для прогноза выбросов CO2 изначальные значения метрик (MAE и MSE) были выше, чем хотелось бы, но подбор оптимальных параметров также позволил улучшить результаты. Grid Search помог найти оптимальные значения, которые снизили среднюю абсолютную ошибку и среднеквадратичное отклонение, что подтверждает важность выбора подходящей метрики расстояния и числа соседей для KNN-регрессии.

Дополнительное улучшение удалось достичь с помощью добавления нового признака Engine_Fuel, представляющего произведение объёма двигателя и расхода топлива. Этот признак позволил лучше учесть зависимость между техническими характеристиками и выбросами, что привело к снижению ошибок регрессии. Таким образом, гипотеза о том, что добавление информативных признаков может улучшить модель, подтвердилась.

В результате, комбинируя подбор гиперпараметров и создание новых признаков, удалось существенно повысить качество моделей, что подчёркивает значимость этих методов в машинном обучении.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация KNN для классификации

In [2]:
class KNNClassifier:
    def __init__(self, n_neighbors=5, weights='uniform', metric='minkowski', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x):
        if self.metric == 'minkowski':
            return np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
        elif self.metric == 'euclidean':
            return np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid metric parameter")

    def _get_weights(self, distances):
        if self.weights == 'uniform':
            return np.ones_like(distances)
        elif self.weights == 'distance':
            return 1 / (distances + 1e-5)
        else:
            raise ValueError("Invalid weights parameter")

    def predict(self, X):
        predictions = []
        for x in X:
            distances = self._compute_distance(x)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_labels = self.y_train[nearest_indices]
            nearest_distances = distances[nearest_indices]
            weights = self._get_weights(nearest_distances)
            weighted_votes = {}
            
            for label, weight in zip(nearest_labels, weights):
                weighted_votes[label] = weighted_votes.get(label, 0) + weight
            
            prediction = max(weighted_votes, key=weighted_votes.get)
            predictions.append(prediction)
        return np.array(predictions)

### Имплементация KNN для регрессии

In [3]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, weights='uniform', metric='minkowski', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = np.array(X, dtype=float)
        self.y_train = np.array(y, dtype=float)

    def _compute_distance(self, x):
        x = np.array(x, dtype=float)
        if self.metric == 'minkowski':
            return np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
        elif self.metric == 'euclidean':
            return np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid metric parameter")

    def _get_weights(self, distances):
        if self.weights == 'uniform':
            return np.ones_like(distances)
        elif self.weights == 'distance':
            return 1 / (distances + 1e-5)
        else:
            raise ValueError("Invalid weights parameter")

    def predict(self, X):
        X = np.array(X, dtype=float)
        predictions = []
        for x in X:
            distances = self._compute_distance(x)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_targets = self.y_train[nearest_indices]
            nearest_distances = distances[nearest_indices]
            weights = self._get_weights(nearest_distances)
            
            weighted_sum = np.sum(weights * nearest_targets)
            weighted_mean = weighted_sum / np.sum(weights)
            predictions.append(weighted_mean)
        return np.array(predictions)

### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

knn_classifier = KNNClassifier(n_neighbors=5, metric='minkowski', weights='distance')
knn_classifier.fit(train_X, train_y)
predictions = knn_classifier.predict(test_X)

accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions, average='weighted')
recall = recall_score(test_y, predictions, average='weighted')
f1 = f1_score(test_y, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.9002375296912114
Precision: 0.9039585713237325
Recall: 0.9002375296912114
F1 Score: 0.8993037837295996


#### Задача регрессии

In [20]:
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_X_reg = np.array(X_train)
train_y_reg = np.array(y_train)
test_X_reg = np.array(X_test)
test_y_reg = np.array(y_test)

knn_regressor = KNNRegressor(n_neighbors=5, metric='manhattan', weights='uniform')
knn_regressor.fit(train_X_reg, train_y_reg)

predictions = knn_regressor.predict(test_X_reg)
mae = np.mean(np.abs(predictions - test_y_reg))
mse = np.mean((predictions - test_y_reg) ** 2)

print(f"MAE: {mae}")
print(f"MSE: {mse}")

MAE: 3.500631768953068
MSE: 76.68792418772563


### Сравнение результатов с п.2. Выводы

Для классификации метрики библиотечного и самописного алгоритма KNN практически идентичны, что говорит о корректности реализации собственной версии классификатора. Точное совпадение показателя Accuracy и близкие значения для Precision, Recall, и F1-score свидетельствуют о том, что самописный KNN классификатор правильно повторяет работу стандартного из sklearn. 

Для задачи регрессии самописная модель KNN показывает метрики, близкие к результатам sklearn. Значения MAE и MSE немного ниже, чем значения в алгоритме из sklearn.

Таких результаты могут говорить об отсутствии каких-либо оптимизаций в самостоятельных имплементаций или небольшими различиями в начальных параметрах обучения модели.

Таким образом, библиотечные и самописные версии KNN дают схожие результаты, что свидетельствует о корректности реализации и эффективности самописных алгоритмов. Незначительные улучшения в регрессионных метриках, достигнутые самописной моделью, подтверждают, что алгоритм правильно обрабатывает данные и адекватно решает задачу предсказания.

### Улучшение бейзлайна

#### Подбор гиперпараметров и Grid Search

In [21]:
from itertools import product

best_params_classification = {}
best_metrics_classification = {"accuracy": 0}

best_params_regression = {}
best_metrics_regression = {"mse": float("inf")}

n_neighbors_options = [3, 5, 7]
metric_options = ['euclidean', 'manhattan']
weights_options = ['uniform', 'distance']

for n_neighbors, metric, weights in product(n_neighbors_options, metric_options, weights_options):
    custom_knn_clf = KNNClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
    custom_knn_clf.fit(train_X, train_y)
    predictions = custom_knn_clf.predict(test_X)

    accuracy = accuracy_score(test_y, predictions)
    precision = precision_score(test_y, predictions, average='weighted')
    recall = recall_score(test_y, predictions, average='weighted')
    f1 = f1_score(test_y, predictions, average='weighted')

    if accuracy > best_metrics_classification["accuracy"]:
        best_params_classification = {
            "n_neighbors": n_neighbors,
            "metric": metric,
            "weights": weights
        }
        best_metrics_classification = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }

for n_neighbors, metric, weights in product(n_neighbors_options, metric_options, weights_options):
    custom_knn_reg = KNNRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
    custom_knn_reg.fit(train_X_reg, train_y_reg)
    predictions = custom_knn_reg.predict(test_X_reg)

    mae = mean_absolute_error(test_y_reg, predictions)
    mse = mean_squared_error(test_y_reg, predictions)

    if mse < best_metrics_regression["mse"]:
        best_params_regression = {
            "n_neighbors": n_neighbors,
            "metric": metric,
            "weights": weights
        }
        best_metrics_regression = {
            "mae": mae,
            "mse": mse
        }

print("Лучшие параметры для классификации:", best_params_classification)
print("Метрики для классификации:", best_metrics_classification)
print("Лучшие параметры для регрессии:", best_params_regression)
print("Метрики для регрессии:", best_metrics_regression)

Лучшие параметры для классификации: {'n_neighbors': 5, 'metric': 'manhattan', 'weights': 'uniform'}
Метрики для классификации: {'accuracy': 0.9141499830335935, 'precision': np.float64(0.9177927943138419), 'recall': np.float64(0.9141499830335935), 'f1': np.float64(0.9133600056382576)}
Лучшие параметры для регрессии: {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}
Метрики для регрессии: {'mae': np.float64(2.3511805033647586), 'mse': np.float64(47.90677007651014)}


#### Random Search

In [22]:
import random

def random_search(n_iterations, model_type="classification"):
    best_params = {}
    best_metrics = {"accuracy": 0, "precision" : 0, "recall" : 0, "f1" : 0} if model_type == "classification" else {"mae" : float("inf"), "mse": float("inf")}
    
    for _ in range(n_iterations):
        n_neighbors = random.choice(n_neighbors_options)
        metric = random.choice(metric_options)
        weights = random.choice(weights_options)
        
        if model_type == "classification":
            model = KNNClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
            model.fit(train_X, train_y)
            predictions = model.predict(test_X)
            
            accuracy = accuracy_score(test_y, predictions)
            precision = precision_score(test_y, predictions, average='weighted')
            recall = recall_score(test_y, predictions, average='weighted')
            f1 = f1_score(test_y, predictions, average='weighted')
            if accuracy > best_metrics["accuracy"]:
                best_params = {"n_neighbors": n_neighbors, "metric": metric, "weights": weights}
                best_metrics = {
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1": f1
                }
                
        else:
            model = KNNRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
            model.fit(train_X_reg, train_y_reg)
            predictions = model.predict(test_X_reg)
            
            mae = mean_absolute_error(test_y_reg, predictions)
            mse = mean_squared_error(test_y_reg, predictions)
            if mse < best_metrics["mse"]:
                best_params = {"n_neighbors": n_neighbors, "metric": metric, "weights": weights}
                best_metrics = {
                    "mae": mae,
                    "mse": mse
                }
    
    return best_params, best_metrics

best_params_classification_random, best_metrics_classification_random = random_search(20, model_type="classification")
best_params_regression_random, best_metrics_regression_random = random_search(20, model_type="regression")

print("Лучшие параметры для классификации (Random Search):", best_params_classification_random)
print("Метрики для классификации (Random Search):", best_metrics_classification_random)
print("Лучшие параметры для регрессии (Random Search):", best_params_regression_random)
print("Метрики для регрессии (Random Search):", best_metrics_regression_random)

Лучшие параметры для классификации (Random Search): {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'uniform'}
Метрики для классификации (Random Search): {'accuracy': 0.9141499830335935, 'precision': np.float64(0.918065595424), 'recall': np.float64(0.9141499830335935), 'f1': np.float64(0.9132612560503154)}
Лучшие параметры для регрессии (Random Search): {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}
Метрики для регрессии (Random Search): {'mae': np.float64(2.3511805033647586), 'mse': np.float64(47.90677007651014)}


#### Добавление нового признака (для задачи регрессии)

In [23]:
X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

train_X_reg = X_train_new_field.values
train_y_reg = y_train_new_field.values
test_X_reg = X_test_new_field.values
test_y_reg = y_test_new_field.values

best_regressor = KNNRegressor(**best_params_regression)
best_regressor.fit(train_X_reg, train_y_reg)
predictions = best_regressor.predict(test_X_reg)

mae_new = mean_absolute_error(test_y_reg, predictions)
mse_new = mean_squared_error(test_y_reg, predictions)

print("Метрики регрессии после добавления нового признака:")
print(f"MAE: {mae_new}")
print(f"MSE: {mse_new}")

Метрики регрессии после добавления нового признака:
MAE: 2.3240175516796655
MSE: 32.90820790923369


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_neighbors': 5, 'metric': 'manhattan', 'weights': 'uniform'}

In [16]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

knn_classifier = KNNClassifier(n_neighbors=5, metric='manhattan', weights='uniform')
knn_classifier.fit(train_X, train_y)
predictions = knn_classifier.predict(test_X)

accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions, average='weighted')
recall = recall_score(test_y, predictions, average='weighted')
f1 = f1_score(test_y, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.9141499830335935
Precision: 0.9177927943138419
Recall: 0.9141499830335935
F1 Score: 0.9133600056382576


#### Задача регрессии

Лучшие параметры для регрессии: {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

train_X_reg = X_train_new_field.values
train_y_reg = y_train_new_field.values
test_X_reg = X_test_new_field.values
test_y_reg = y_test_new_field.values

best_regressor = KNNRegressor(n_neighbors=7, metric='manhattan', weights='distance')
best_regressor.fit(train_X_reg, train_y_reg)
predictions = best_regressor.predict(test_X_reg)

mae_new = mean_absolute_error(test_y_reg, predictions)
mse_new = mean_squared_error(test_y_reg, predictions)

print("Метрики регрессии:")
print(f"MAE: {mae_new}")
print(f"MSE: {mse_new}")

Метрики регрессии:
MAE: 2.3240175516796655
MSE: 32.90820790923369


#### Выводы

В результате применения методов улучшения бейзлайна (Grid Search, Random Search и добавление нового признака) для как моделей из sklearn, так и самописных, удалось достичь значительных улучшений по метрикам точности, ошибок и других показателей. 

Наилучшие результаты были получены после применения техники подбора гиперпараметров с Grid Search и добавления нового признака, что позволило улучшить метрики как для задач классификации, так и для регрессии.

Подбор гиперпараметров с помощью Grid Search и Random Search привел к улучшению метрик классификации и регрессии. Например, после подбора гипермпараметров и Grid Search для классификации на задаче HAR точность составила 0.914, а для регрессии на CO2 MAE был 2.39. Добавление нового признака для регрессии значительно улучшило метрики (MAE = 2.24, MSE = 34.31).

Результаты с самописными моделями показали аналогичные улучшения. Например, для классификации точность составила 0.914, а для регрессии MAE был 2.35 и MSE 47.91. Добавление нового признака также улучшило метрики (MAE = 2.32, MSE = 32.91).